In [1]:
import numpy as np

In [2]:
# desired correlation matrix
cor_matrix = np.array([[1.0, 0.8],
                       [0.8, 1.0]])

L = np.linalg.cholesky(cor_matrix)

In [3]:
import numpy as np
import scipy.linalg

def var_to_tsdata(A, SIG, m, N=1, mtrunc=None, decayfac=100):
    def genvar(A, E, trunc=0):
        n1, n2, p = A.shape
        n, m = E.shape
        assert n1 == n2 and n1 == n, 'VAR coefficients blocks not square or do not match residuals covariance matrix size'
        assert trunc >= 0 and trunc < m, 'bad truncation'

        X = E.copy()
        for t in range(p, m):
            for k in range(1, p+1):
                X[:, t] += A[:, :, k-1] @ X[:, t-k]

        if trunc > 0:
            X = X[:, trunc:]
            E = E[:, trunc:]

        return X, E

    if mtrunc is None:
        rho = max(abs(np.linalg.eig(A)[0]))
        assert rho < 1, 'unstable VAR'
        mtrunc = int(round((np.log(np.finfo(float).eps) - decayfac) / np.log(rho)))

    C = scipy.linalg.cholesky(SIG, lower=True)
    n = A.shape[0]

    if N > 1:
        X = np.zeros((n, m, N))
        E = np.zeros((n, m, N))
        for r in range(N):
            X[:, :, r], E[:, :, r] = genvar(A, C @ np.random.randn(n, m + mtrunc), mtrunc)
    else:
        X, E = genvar(A, C @ np.random.randn(n, m + mtrunc), mtrunc)

    return X, E, mtrunc


# #VAR coefficients matrix
# A = np.array([[[1,0,1],[0,1,0]],[[.5,0,.5],[0,.1,0]]])
# #residuals covariance matrix. use identity matrix, same shape as a, for white noise
# SIG = np.eye(A.shape[0])
# sampling_rate = 2048
# m = sampling_rate * 4 # in sec
# N = 1
# mtrunc = sampling_rate
# decayfac = 100

# X, E, mtrunc = var_to_tsdata(A, SIG, m, N, mtrunc, decayfac)

In [4]:
# #VAR coefficients matrix
# A = np.array([[[1,0,1],[0,1,0]],[[.5,0,.5],[0,.1,0]]])
# #residuals covariance matrix. use identity matrix, same shape as a, for white noise
# SIG = np.eye(A.shape[0])
# sampling_rate = 2048
# m = sampling_rate * 4 # in sec
# N = 1
# mtrunc = sampling_rate
# decayfac = 100

# X, E, mtrunc = var_to_tsdata(A, SIG, m, N, mtrunc, decayfac)

In [5]:
# E

In [6]:
import matplotlib.pyplot as plt
# plt.plot(X.T)

In [7]:
fs = 2048  # Sampling rate (Hz)
T = 150  # Length of epochs (s)

import numpy as np
from scipy.stats import chi2, norm
import math

# Set the seed for reproducibility
np.random.seed(0)

# Define the number of iterations for the simulation
n_iterations = fs*T

# Preallocate the arrays for the x variables
x1 = np.zeros(n_iterations)
x2 = np.zeros(n_iterations)
x3 = np.zeros(n_iterations)
x4 = np.zeros(n_iterations)
x5 = np.zeros(n_iterations)

# Define the rate lambda for the exponential distribution
lambda_rate = 2

# Generate the noise processes e1t, e2t, e3t, e4t, e5t
e1 = np.random.exponential(scale=1 / lambda_rate, size=n_iterations)
e2 = chi2.rvs(df=1, size=n_iterations)
e3 = norm.rvs(scale=1, size=n_iterations)  # Gaussian with mean 0, std 1
e4 = norm.rvs(scale=1, size=n_iterations)  # Gaussian with mean 0, std 1
e5 = norm.rvs(scale=1, size=n_iterations)  # Gaussian with mean 0, std 1

for t in range(1, n_iterations):
    # Generate the x variables based on the given equations
    x1[t] = e1[t]
    x2[t] = e2[t]
    x3[t] = 0.8 * x2[t] + e3[t]
    x4[t] = 0.7 * x1[t] * (math.pow(x1[t],2) - 1) * np.exp((-math.pow(x1[t],2)) / 2) + e4[t]
    x5[t] = 0.3 * x2[t] + 0.05 * math.pow(x2[t],2) + e5[t]

# After the loop, x1t, x2t, x3t, x4t, and x5t contain the simulation data.

PLOT_VAR_NAMES = np.arange(5) + 1
PLOT_VAR_IDXS = np.arange(5)

data = np.array([x1, x2, x3, x4, x5]).T

In [8]:
data[1]

array([ 0.62796538,  0.02411127, -0.30792015, -0.32738685, -0.02491688])

In [9]:
data[-1]

array([0.0096291 , 2.54564524, 2.69902039, 1.07702047, 1.73119984])